# Two-View 3D Reconstruction

This notebook implements a simple two-view Structure from Motion approach to create a 3D reconstruction from two images (01.jpg and 02.jpg).

## Setup and Requirements

In [ ]:
# Install required packages
!pip install open3d networkx matplotlib tqdm opencv-python

In [ ]:
import cv2
import open3d as o3d
import os
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# For displaying Open3D visualizations in notebooks
from google.colab.patches import cv2_imshow
import base64
from IPython.display import HTML

def display_open3d_to_notebook(vis, width=900, height=600):
    vis.update_renderer()
    image = vis.capture_screen_float_buffer()
    image_array = np.asarray(image)
    image_array = (image_array * 255).astype(np.uint8)
    image_array = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
    _, encoded_img = cv2.imencode('.png', image_array)
    encoded_img = base64.b64encode(encoded_img)
    html = f'<img src="data:image/png;base64,{encoded_img.decode()}" width="{width}" height="{height}"/>'
    return HTML(html)

## 1. Data Loading

First, let's load our two specific images: 01.jpg and 02.jpg.

In [ ]:
!mkdir images
!wget -P /content/images https://raw.githubusercontent.com/VarunBurde/Prague_ml/main/images/01.jpg
!wget -P /content/images https://raw.githubusercontent.com/VarunBurde/Prague_ml/main/images/02.jpg
!wget -O /content/K.txt https://raw.githubusercontent.com/VarunBurde/Prague_ml/main/K.txt

In [ ]:
# For Colab: Upload images
from google.colab import files
import shutil

# Set the paths for our specific images
img_path = "/content/images"
image_paths = [os.path.join(img_path, "01.jpg"), os.path.join(img_path, "02.jpg")]
K = np.loadtxt("/content/K.txt")

plt.figure(figsize=(15, 7))
for i, path in enumerate(image_paths):
    plt.subplot(1, 2, i+1)
    img = cv2.imread(path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.title(f"Image {i+1}: {os.path.basename(path)}")
plt.tight_layout()
plt.show()

## 2. Load Images and Extract Features

Now we'll load our two images and extract feature points using SIFT.

In [ ]:
# Create feature detectors with optimized parameters
sift = cv2.SIFT_create(
    nfeatures=5000,  # Increased from 0
    contrastThreshold=0.02,  # Lowered for more features
    edgeThreshold=20,  # Increased for more edge features
    sigma=1.6
)

# Add ORB detector for additional features
orb = cv2.ORB_create(
    nfeatures=5000,
    scaleFactor=1.2,
    nlevels=8,
    edgeThreshold=31,
    firstLevel=0,
    WTA_K=2,
    patchSize=31
)

In [ ]:
def extract_features(img_rgb):
    """Extract features using multiple detectors and image processing"""
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    img_eq = cv2.equalizeHist(img_gray)
    img_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)).apply(img_gray)

    # SIFT features from different image versions
    kp1, des1 = sift.detectAndCompute(img_gray, None)
    kp2, des2 = sift.detectAndCompute(img_eq, None)
    kp3, des3 = sift.detectAndCompute(img_clahe, None)

    # ORB features
    kp4, des4 = orb.detectAndCompute(img_gray, None)

    # Combine all features
    keypoints = kp1 + kp2 + kp3 + kp4
    descriptors = []

    if des1 is not None:
        descriptors.append(des1)
    if des2 is not None:
        descriptors.append(des2)
    if des3 is not None:
        descriptors.append(des3)
    if des4 is not None:
        # Convert ORB descriptors to float32 for compatibility
        des4 = np.float32(des4)
        if des4.shape[1] != 128:  # SIFT descriptor length
            des4 = cv2.resize(des4, (128, des4.shape[0]))
        descriptors.append(des4)

    if descriptors:
        descriptors = np.vstack(descriptors)
    else:
        descriptors = np.array([])

    return keypoints, descriptors

# Extract features for all images
all_kp = []
all_des = []

for i, img_rgb in enumerate(images_rgb):
    kp, des = extract_features(img_rgb)
    all_kp.append(kp)
    all_des.append(des)
    print(f"Image {i+1} ({os.path.basename(image_paths[i])}): Detected {len(kp)} keypoints")

### Visualize Detected Features

Let's visualize the keypoints detected in both images.

In [ ]:
# Display keypoints on both images
plt.figure(figsize=(15, 7))
for i in range(2):
    plt.subplot(1, 2, i+1)
    img_with_kp = cv2.drawKeypoints(images_rgb[i], all_kp[i], None, 
                                    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.imshow(img_with_kp)
    plt.title(f"Image {i+1} ({os.path.basename(image_paths[i])}): {len(all_kp[i])} keypoints")
plt.tight_layout()
plt.show()

## 3. Match Features Between Images

Now we'll match features between our two images to find correspondences.

In [ ]:
# Improved FLANN matcher setup
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=100)  # More checks for better accuracy
flann = cv2.FlannBasedMatcher(index_params, search_params)

# Function to perform feature matching with cross-check
def match_features(des1, des2, lowes_ratio=0.75, cross_check=True):
    # Forward matching (img1 -> img2)
    if des1 is None or des2 is None or len(des1) == 0 or len(des2) == 0:
        return []
    
    matches12 = flann.knnMatch(des1, des2, k=2)
    good_matches12 = []
    for m, n in matches12:
        if m.distance < lowes_ratio * n.distance:
            good_matches12.append(m)
    
    if not cross_check:
        return good_matches12
    
    # Backward matching (img2 -> img1) for cross-check
    matches21 = flann.knnMatch(des2, des1, k=2)
    good_matches21 = []
    for m, n in matches21:
        if m.distance < lowes_ratio * n.distance:
            good_matches21.append(m)
    
    # Cross-check: keep matches that are consistent in both directions
    cross_matches = []
    for match12 in good_matches12:
        for match21 in good_matches21:
            # Check if match is consistent in both directions
            if match12.queryIdx == match21.trainIdx and match12.trainIdx == match21.queryIdx:
                cross_matches.append(match12)
                break
    
    return cross_matches

# Get matched points from keypoints and matches
def get_matched_points(kp1, kp2, matches):
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    return pts1, pts2

In [ ]:
# Match features between our two images (01.jpg and 02.jpg)
idx1, idx2 = 0, 1  # Using image indices 0 and 1 (01.jpg and 02.jpg)
good_matches = match_features(all_des[idx1], all_des[idx2], lowes_ratio=0.75)

print(f"Found {len(good_matches)} good matches between {os.path.basename(image_paths[0])} and {os.path.basename(image_paths[1])}")

# Display matches for verification
match_img = cv2.drawMatches(
    images_rgb[idx1], all_kp[idx1], 
    images_rgb[idx2], all_kp[idx2], 
    good_matches[:100], None, 
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)
plt.figure(figsize=(15, 10))
plt.imshow(match_img)
plt.title(f'Matches between {os.path.basename(image_paths[0])} and {os.path.basename(image_paths[1])}')
plt.show()

## 4. Two-View Reconstruction

Now we'll use the matched features to perform 3D reconstruction.

In [ ]:
# Initialize camera poses (R|t) for each image
# First camera (01.jpg) is at origin with identity rotation
camera_poses = [np.hstack((np.eye(3), np.zeros((3, 1))))]
camera_matrices = [K @ camera_poses[0]]

# Structure to store 3D points and their 2D observations
point_cloud = []
point_colors = []

pts1, pts2 = get_matched_points(all_kp[idx1], all_kp[idx2], good_matches)

# Calculate essential matrix with robust parameters
E, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.RANSAC, prob=0.999, threshold=2.0)

# Recover pose for second camera (02.jpg)
_, R, t, mask = cv2.recoverPose(E, pts1, pts2, K, mask=mask)

# Set second camera pose
camera_poses.append(np.hstack((R, t)))
camera_matrices.append(K @ camera_poses[1])

print(f"Camera 1 (01.jpg) pose:\n{camera_poses[0]}")
print(f"\nCamera 2 (02.jpg) pose:\n{camera_poses[1]}")

# Filter points using mask
mask = mask.ravel() == 1
pts1_good = pts1[mask]
pts2_good = pts2[mask]

print(f"\nUsing {np.sum(mask)} inlier matches for triangulation")

In [ ]:
# Triangulate points
pts1_good = pts1_good.reshape(-1, 2).T
pts2_good = pts2_good.reshape(-1, 2).T
points_4D = cv2.triangulatePoints(camera_matrices[0], camera_matrices[1], pts1_good, pts2_good)
points_3D = points_4D / points_4D[3]  # Convert to Cartesian
points_3D = points_3D[:3, :].T

# Filter points by depth
valid_points = []
valid_colors = []

for i in range(points_3D.shape[0]):
    point = points_3D[i]
    # Check if point is in front of both cameras
    if point[2] > 0:
        # Get corresponding 2D points
        x1, y1 = pts1_good[:, i]
        
        # Extract color from first image
        x1_int, y1_int = int(round(x1)), int(round(y1))
        if 0 <= x1_int < images_rgb[idx1].shape[1] and 0 <= y1_int < images_rgb[idx1].shape[0]:
            color = images_rgb[idx1][y1_int, x1_int] / 255.0
            valid_points.append(point)
            valid_colors.append(color)

point_cloud = np.array(valid_points)
point_colors = np.array(valid_colors)

print(f"Reconstructed point cloud has {len(point_cloud)} points")

### Visualize Point Cloud
Let's visualize the 3D point cloud reconstructed from our two images.

In [ ]:
# Create Open3D point cloud for visualization
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud)
pcd.colors = o3d.utility.Vector3dVector(point_colors)

# Remove outliers for cleaner visualization
pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

In [ ]:
def draw_geometries(geometries):
    graph_objects = []

    for geometry in geometries:
        geometry_type = geometry.get_geometry_type()

        if geometry_type == o3d.geometry.Geometry.Type.PointCloud:
            points = np.asarray(geometry.points)
            colors = None
            if geometry.has_colors():
                colors = np.asarray(geometry.colors)
            elif geometry.has_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.normals) * 0.5
            else:
                geometry.paint_uniform_color((1.0, 0.0, 0.0))
                colors = np.asarray(geometry.colors)

            scatter_3d = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=1, color=colors))
            graph_objects.append(scatter_3d)

        if geometry_type == o3d.geometry.Geometry.Type.TriangleMesh:
            triangles = np.asarray(geometry.triangles)
            vertices = np.asarray(geometry.vertices)
            colors = None
            if geometry.has_triangle_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.triangle_normals) * 0.5
                colors = tuple(map(tuple, colors))
            else:
                colors = (1.0, 0.0, 0.0)

            mesh_3d = go.Mesh3d(x=vertices[:,0], y=vertices[:,1], z=vertices[:,2], i=triangles[:,0], j=triangles[:,1], k=triangles[:,2], facecolor=colors, opacity=0.50)
            graph_objects.append(mesh_3d)

    fig = go.Figure(
        data=graph_objects,
        layout=dict(
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False)
            )
        )
    )
    fig.show()

In [ ]:
o3d.visualization.draw_geometries = draw_geometries # replace function
o3d.visualization.draw_geometries([pcd])

## 5. Multi-View Reconstruction

Now let's extend our reconstruction to handle multiple views and compare with the two-view result.

In [ ]:
# Download additional images
!wget -P /content/images https://raw.githubusercontent.com/VarunBurde/Prague_ml/main/images/03.jpg
!wget -P /content/images https://raw.githubusercontent.com/VarunBurde/Prague_ml/main/images/04.jpg

# Update image paths for multi-view
multi_image_paths = sorted(glob(os.path.join(img_path, "*.jpg")))
print(f"Total images for multi-view reconstruction: {len(multi_image_paths)}")

In [ ]:
def triangulate_points(P1, P2, pts1, pts2):
    """Triangulate 3D points from 2D correspondences"""
    # Ensure points are 2xN
    pts1 = np.ascontiguousarray(pts1.reshape(-1, 2).T)
    pts2 = np.ascontiguousarray(pts2.reshape(-1, 2).T)

    # Triangulate
    points_4d = cv2.triangulatePoints(P1, P2, pts1, pts2)
    points_3d = (points_4d[:3] / points_4d[3]).T

    return points_3d

def check_pose_scale(R, t, pts1, pts2, K):
    """Check if recovered pose has reasonable scale"""
    P1 = K @ np.hstack((np.eye(3), np.zeros((3, 1))))
    P2 = K @ np.hstack((R, t))
    
    try:
        points_3d = triangulate_points(P1, P2, pts1, pts2)
        depths = points_3d[:, 2]
        if len(depths[depths > 0]) == 0:
            return False
            
        median_depth = np.median(depths[depths > 0])
        point_spread = np.std(points_3d, axis=0)
        
        return 0.1 < median_depth < 100 and np.all(point_spread < 50)
    except:
        return False

def refine_pose(R, t, pts1, pts2, K):
    """Refine pose estimation using PnP"""
    try:
        P1 = K @ np.hstack((np.eye(3), np.zeros((3, 1))))
        P2 = K @ np.hstack((R, t))
        points_3d = triangulate_points(P1, P2, pts1, pts2)

        # Use PnP to refine pose
        success, R_vec, t_refined, inliers = cv2.solvePnPRansac(
            points_3d,
            pts2.reshape(-1, 1, 2),
            K,
            None,
            flags=cv2.SOLVEPNP_ITERATIVE,
            confidence=0.99,
            reprojectionError=8.0
        )

        if not success:
            return R, t, None

        R_refined, _ = cv2.Rodrigues(R_vec)
        return R_refined, t_refined, inliers
    except:
        return R, t, None

In [ ]:
def estimate_relative_pose(pts1, pts2, K, min_inliers=15):
    """Improved relative pose estimation"""
    if pts1.shape[0] < min_inliers:
        return None, None, None, 0

    # Try multiple RANSAC thresholds
    thresholds = [0.001, 0.005, 0.01, 0.02]
    best_result = None
    best_inliers = 0

    for threshold in thresholds:
        try:
            # Normalize points
            pts1_norm = cv2.undistortPoints(pts1.reshape(-1, 1, 2), K, None)
            pts2_norm = cv2.undistortPoints(pts2.reshape(-1, 1, 2), K, None)

            # Estimate essential matrix
            E, mask = cv2.findEssentialMat(
                pts1_norm,
                pts2_norm,
                np.eye(3),
                method=cv2.RANSAC,
                prob=0.999,
                threshold=threshold
            )

            if E is None:
                continue

            # Recover pose
            _, R, t, mask = cv2.recoverPose(E, pts1_norm, pts2_norm, np.eye(3), mask=mask)
            inliers = np.sum(mask)

            if inliers > best_inliers:
                best_inliers = inliers
                best_result = (R, t, mask, inliers)

        except Exception as e:
            print(f"Error in pose estimation: {e}")
            continue

    if best_result is None:
        return None, None, None, 0

    R, t, mask, inliers = best_result
    
    # Refine pose
    good_pts1 = pts1[mask.ravel()==1]
    good_pts2 = pts2[mask.ravel()==1]
    R_refined, t_refined, _ = refine_pose(R, t, good_pts1, good_pts2, K)

    return R_refined, t_refined, mask, inliers

In [ ]:
def process_image_pair(img1_idx, img2_idx, all_images, all_descriptors, all_keypoints, K, camera_poses, camera_matrices=None):
    """Process a pair of images for reconstruction"""
    if img1_idx >= len(camera_poses):
        print(f"Warning: Camera pose not available for image {img1_idx}")
        return None, None, None, -1

    best_pose = None
    best_points = None
    best_colors = None
    best_score = -1

    # Get camera matrix for first image
    P1 = K @ camera_poses[img1_idx]

    # Try different matching parameters
    for ratio in [0.8, 0.85, 0.75]:
        try:
            matches = match_features(all_descriptors[img1_idx], all_descriptors[img2_idx], lowes_ratio=ratio)
            if len(matches) < 100:  # Increased minimum matches requirement
                continue

            pts1, pts2 = get_matched_points(all_keypoints[img1_idx], all_keypoints[img2_idx], matches)
            R, t, mask, inliers = estimate_relative_pose(pts1, pts2, K)

            if R is None:
                continue

            # Convert to global coordinates
            R_prev = camera_poses[img1_idx][:, :3]
            t_prev = camera_poses[img1_idx][:, 3:]
            R_new = R @ R_prev
            t_new = R @ t_prev + t

            # Create new camera matrix
            P2 = K @ np.hstack((R_new, t_new))

            # Triangulate points
            good_pts1 = pts1[mask.ravel()==1]
            good_pts2 = pts2[mask.ravel()==1]
            points_3d = triangulate_points(P1, P2, good_pts1, good_pts2)

            # Score this pose
            valid_points = points_3d[points_3d[:, 2] > 0]
            if len(valid_points) < 50:  # Increased minimum points requirement
                continue
                
            score = inliers * (1 - np.mean(np.abs(valid_points[:, 2])))

            if score > best_score:
                best_score = score
                best_pose = np.hstack((R_new, t_new))
                best_points = points_3d
                
                # Get colors from first image
                colors = []
                for pt, (x, y) in zip(points_3d, good_pts1):
                    if 0 <= int(y) < all_images[img1_idx].shape[0] and 0 <= int(x) < all_images[img1_idx].shape[1]:
                        color = all_images[img1_idx][int(y), int(x)] / 255.0
                        colors.append(color)
                best_colors = np.array(colors)
                
        except Exception as e:
            print(f"Error processing image pair {img1_idx}-{img2_idx}: {str(e)}")
            continue

    return best_pose, best_points, best_colors, best_score

In [ ]:
def reconstruct_multiview(image_paths, K):
    """Multi-view reconstruction pipeline"""
    # Initialize variables
    all_images = []
    all_keypoints = []
    all_descriptors = []
    camera_poses = []
    camera_matrices = []
    points_3d = []
    point_colors = []

    # Load and extract features
    for path in image_paths:
        img = cv2.imread(path)
        if img is None:
            print(f"Failed to load image: {path}")
            continue
            
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        kp, des = extract_features(img_rgb)
        
        all_images.append(img_rgb)
        all_keypoints.append(kp)
        all_descriptors.append(des)
        print(f"Extracted {len(kp)} features from {os.path.basename(path)}")

    if len(all_images) < 2:
        raise ValueError("Need at least 2 valid images")

    # Initialize first camera
    first_pose = np.hstack((np.eye(3), np.zeros((3, 1))))
    camera_poses.append(first_pose)
    camera_matrices.append(K @ first_pose)

    # Sequential reconstruction
    for i in range(1, len(all_images)):
        best_pose = None
        best_points = None
        best_colors = None
        best_score = -1

        # Try matching with previous views, prioritizing recent ones
        prev_indices = list(range(max(0, i-3), i))[::-1]  # Reverse order to try recent first
        
        for j in prev_indices:
            if j >= len(camera_poses):  # Skip if we don't have the camera pose
                continue

            pose, pts3d, colors, score = process_image_pair(
                j, i, all_images, all_descriptors, all_keypoints,
                K, camera_poses, camera_matrices
            )

            if pose is not None and score > best_score:
                best_pose = pose
                best_points = pts3d
                best_colors = colors
                best_score = score

                # If we got a good result from the most recent frame, stop searching
                if j == i-1 and score > 100:  # Threshold for a 'good enough' result
                    break

        if best_pose is not None:
            camera_poses.append(best_pose)
            camera_matrices.append(K @ best_pose)
            if best_points is not None and best_colors is not None:
                points_3d.extend(best_points)
                point_colors.extend(best_colors)
                print(f"Added image {i} with {len(best_points)} points (score: {best_score:.2f})")
        else:
            print(f"Warning: Could not find good pose for image {i}")
            # Add dummy pose to maintain indexing
            dummy_pose = camera_poses[-1].copy()  # Copy last pose
            camera_poses.append(dummy_pose)
            camera_matrices.append(K @ dummy_pose)

    if len(points_3d) == 0 or len(point_colors) == 0:
        raise ValueError("Failed to reconstruct any 3D points")

    return np.array(points_3d), np.array(point_colors), camera_poses

In [ ]:
# Perform multi-view reconstruction
mv_points, mv_colors, mv_cameras = reconstruct_multiview(multi_image_paths, K)

# Create point clouds for comparison
pcd_two_view = o3d.geometry.PointCloud()
pcd_two_view.points = o3d.utility.Vector3dVector(point_cloud)
pcd_two_view.colors = o3d.utility.Vector3dVector(point_colors)

pcd_multi_view = o3d.geometry.PointCloud()
pcd_multi_view.points = o3d.utility.Vector3dVector(mv_points)
pcd_multi_view.colors = o3d.utility.Vector3dVector(mv_colors)

# Clean both point clouds
pcd_two_view, _ = pcd_two_view.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
pcd_multi_view, _ = pcd_multi_view.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

print(f"Two-view reconstruction: {len(pcd_two_view.points)} points")
print(f"Multi-view reconstruction: {len(pcd_multi_view.points)} points")

### Comparison of Two-View vs Multi-View Reconstruction

Below we visualize both reconstructions side by side. The multi-view reconstruction typically provides:
- More complete coverage of the scene
- Better handling of occlusions
- More accurate point positions due to multiple observations
- Higher point density in well-observed regions

In [ ]:
# Visualize two-view reconstruction
print("Two-view reconstruction:")
o3d.visualization.draw_geometries([pcd_two_view])

# Visualize multi-view reconstruction
print("\nMulti-view reconstruction:")
o3d.visualization.draw_geometries([pcd_multi_view])